In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

df = pd.read_csv(r"C:\Users\johan\Thesis\ThesisCode\lorenz_windows.csv")

print(df.head(5))


        x_t0       y_t0       z_t0       x_t1       y_t1       z_t1  \
0  -9.786929 -15.038520  20.533978 -10.577554 -15.833834  22.044518   
1 -10.577554 -15.833834  22.044518 -11.355439 -16.429680  23.787565   
2 -11.355439 -16.429680  23.787565 -12.089270 -16.752077  25.722356   
3 -12.089270 -16.752077  25.722356 -12.742066 -16.732771  27.779601   
4 -12.742066 -16.732771  27.779601 -13.273643 -16.321915  29.861869   

        x_t2       y_t2       z_t2       x_t3  ...      z_t17     x_t18  \
0 -11.355439 -16.429680  23.787565 -12.089270  ...  31.718965 -5.990126   
1 -12.089270 -16.752077  25.722356 -12.742066  ...  30.552253 -5.236159   
2 -12.742066 -16.732771  27.779601 -13.273643  ...  29.401553 -4.570703   
3 -13.273643 -16.321915  29.861869 -13.644591  ...  28.281191 -3.994244   
4 -13.644591 -15.501392  31.850541 -13.821372  ...  27.198773 -3.503766   

      y_t18      z_t18     x_t19     y_t19      z_t19    x_next    y_next  \
0 -0.680717  30.552253 -5.236159 -0.504723  2

In [ ]:
feature_col = [c for c in df.columns if c.endswith("_t0")]
target_col = [c for c in df.columns if c.endswith("_next")]

X = df[feature_col].values
y = df[target_col].values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 123)

In [ ]:
batch_size = 32
def make_dataset(X_arr, y_arr, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X_arr, y_arr))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X_arr))
    return ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

ds_train = make_dataset(X_train, y_train)
ds_val   = make_dataset(X_val,   y_val,   shuffle=False)

In [ ]:
# 5. Define a minimal FFNN
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1],)),   # e.g. (3,) for last-state
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(3)                            # predict next x,y,z
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss="mse",
    metrics=["mae"]
)


In [ ]:

# 6. Quick “smoke test” fit
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=5
)


In [1]:
import tensorflow as tf

# List all physical GPUs
gpus = tf.config.list_physical_devices('GPU')
print("GPUs detected:", gpus)

# List all CPUs (just for completeness)
cpus = tf.config.list_physical_devices('CPU')
print("CPUs detected:", cpus)


GPUs detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
CPUs detected: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
